In [91]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 

In [4]:
class Advertisment:
    
    def __init__(self,title,price,link,status='Avaliable'):
        self.title = title
        self.price = price
        self.status = status
        self.link  = link
    
    def getStatus(self):
        return self.status
    def getPrice(self):
        return self.price
    def getTitle(self):
        return self.title
    def getLink(self):
        return self.link
    
    def toString(self):
        return f'{self.title},{self.price},{self.status},{self.link}\n'
    

In [99]:
class Main:
    
    def __init__(self,url):
        self.parsed_ads = []
        self.raw = []
        self.url = url
        self.urls = []
        self.filename=""
    
    def send_email(self):
        fromaddr = "FROM_EMAIL@gmail.com"
        toaddr = "TO_EMAIL@gmail.com"

        # instance of MIMEMultipart 
        msg = MIMEMultipart() 

        # storing the senders email address   
        msg['From'] = fromaddr 

        # storing the receivers email address  
        msg['To'] = toaddr 

        # storing the subject  
        msg['Subject'] = "Automatic Email"

        # string to store the body of the mail 
        body = "YOUR_BODY"

        # attach the body with the msg instance 
        msg.attach(MIMEText(body, 'plain')) 

        # open the file to be sent  
        attachment = open("YOUR_FILE_PATH"+self.filename, "rb") 

        # instance of MIMEBase and named as p 
        p = MIMEBase('application', 'octet-stream') 

        # To change the payload into encoded form 
        p.set_payload((attachment).read()) 

        # encode into base64 
        encoders.encode_base64(p) 

        p.add_header('Content-Disposition', "attachment; filename= %s" % self.filename) 

        # attach the instance 'p' to instance 'msg' 
        msg.attach(p) 

        # creates SMTP session 
        s = smtplib.SMTP('smtp.gmail.com', 587) 

        # start TLS for security 
        s.starttls() 

        # Authentication 
        #myPassword = input('Please enter the password')
        s.login(fromaddr, "YOUR_PASSWORD") 

        # Converts the Multipart msg into a string 
        text = msg.as_string() 

        # sending the mail 
        s.sendmail(fromaddr, toaddr, text) 

        # terminating the session 
        s.quit() 
    
    #GE
    def generateUrl(self):
        if len(self.urls) != 0 :
            self.urls.clear()
        for i in range(5):
            self.urls.append(f"https://www.qatarliving.com/items/computers-tech?page={i}&search_input=monitor")
    
    
    def getStatus(self,one_obj):
        data=''
        try :
            if one_obj.find('div',attrs={'class':'b-card--el-header'}).find('span',attrs={'class':'b-card--el-featured-label'}) == None :     
                return 'Avaliable'
        except :
            return 'Sold'
    
    def parser(self):
        for one_link in self.urls:
            self.myreq = requests.get(one_link)
            self.soup = BeautifulSoup(self.myreq.content,'html.parser')
            main = self.soup.find('main',attrs={'class':'b-main b-main-mod-right b-main--mix-p-properties-list'})
            ads = main.find('div',attrs={'class':'row'}).find('div',attrs={'class':'col-sm-8'}).find('div',attrs={'id':'classified_search_results'}).find_all('span',attrs={'class':'b-card b-card-mod-h item'})
            for one_ad in ads :
                try:
                    self.raw.append(one_ad)
                    status = self.getStatus(one_ad)
                    price = one_ad.find('div',attrs={'class':'b-card--el-header'}).find('p',attrs={'class':'b-card--el-deposit-val'}).get_text().replace(',','')
                    title = one_ad.find('div',attrs={'class':'b-card--el-details'}).find('a').find('p').get_text().replace(',','')
                    link = one_ad.get('data-entity-url')
                    new_advertisment = Advertisment(title,price,link,status)
                    self.parsed_ads.append(new_advertisment)
                except Exception as e:
                    #print(e)
                    print('')
            time.sleep(3)
    
    def writeResults(self):
        now = datetime.now()
        time_now = now.strftime("%d,%m,%Y,%H,%M,%S")
        self.filename = f'ads{time_now}.csv'
        with open(f'ads{time_now}.csv','w',encoding="utf-8") as file :
            file.write('Title,Price,Status,link\n')
            for ad in self.parsed_ads:
                file.write(ad.toString())
            file.close()
    
    
    def main_controller(self):
        #PUT THE CODE OF RECURRANCE HERE 
        self.generateUrl()    
        self.parser()
        self.writeResults()
        self.send_email()

        
    
    
    
    

In [100]:
started = Main(url = "https://www.qatarliving.com/items/computers-tech?search_input=monitor")
started.main_controller()